# Bash function

In [1]:
def bash(string,name,queue):
    script = "%s.sh"%name
    with open(script, "w") as text_file:
        text_file.write(string)
    !chmod +x $script
    !qsub -q $queue $script
    !rm $script
    
ws = '/beegfs/work/graaf20/Rusitec_meta'

#Create list of file indexes to access it later
import os
import pandas as pd
md = pd.read_csv('metadata.tsv',index_col=0,sep='\t')
bss = md.BS.unique()
!rm -r $ws/Pooled/.ipynb_checkpoints
reads = [f.split('_')[0] for f in os.listdir(ws+'/Pooled') if 'pooled' not in f]
reads = list(set(reads))
reads = [str(x) for x in sorted([int(i) for i in reads])]
print(reads)

rm: cannot remove ‘/beegfs/work/graaf20/Rusitec_meta/Pooled/.ipynb_checkpoints’: No such file or directory
['1', '4', '9', '11', '13', '15', '17', '19', '21', '23', '25', '27', '29', '31', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '80', '83', '88', '90', '92', '94', '96', '98', '100', '102', '104', '106', '108', '110', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134']


In [ ]:
!rm -rf Cat_reads ReadQC

In [2]:
!git clone https://github.com/bxlab/metaWRAP.git

fatal: destination path 'metaWRAP' already exists and is not an empty directory.


# Databases

In [5]:
string = '''
#PBS -l nodes=1:ppn=24
#PBS -l walltime=48:00:00
#PBS -l mem=256gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
kraken2-build --download-library archaea --db %s --no-masking
kraken2-build --download-library bacteria --db %s --no-masking
kraken2-build --download-library protozoa --db %s --no-masking
kraken2-build --download-library fungi --db %s --no-masking
kraken2-build --download-taxonomy --db %s
kraken2-build --build --threads 24 --db %s
'''
KD = ws+'/Databases/Kraken2'
!mkdir $KD
bash(string%(KD,KD,KD,KD,KD,KD),'Krk2_db','smp')


9734905


In [6]:
string = '''
#PBS -l nodes=1:ppn=24
#PBS -l walltime=48:00:00
#PBS -l mem=256gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
kraken2-build --build --threads 24 --db %s
'''
KD = ws+'/Databases/Kraken2'
!mkdir $KD
bash(string%(KD),'Krk2_db','smp')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Databases/Kraken2’: File exists
9735450


In [6]:
ref = ws+'/Databases/Cattle_db/GCF_002263795.1.fna.gz'
unz = ws+'/Databases/Cattle_db/GCF_002263795.1.fna'
!zcat $ref > $unz

In [3]:
string = '''
#PBS -l nodes=1:ppn=4
#PBS -l walltime=24:00:00
#PBS -l mem=8gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
bmtool -d %s -o %s
'''
ref = ws+'/Databases/Cattle_db/GCF_002263795.1.fna'
srp = ws+'/Databases/Cattle_db/GCF_002263795.srprism'
bmt = ws+'/Databases/Cattle_db/GCF_002263795.bitmask'
bash(string%(ref,bmt),'bmt_db','short')

9735448


In [9]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=12:00:00
#PBS -l mem=100gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
srprism mkindex -i %s -o %s -M 100000
'''
ref = ws+'/Databases/Cattle_db/GCF_002263795.1.fna'
srp = ws+'/Databases/Cattle_db/GCF_002263795.srprism'
bmt = ws+'/Databases/Cattle_db/GCF_002263795.bitmask'
bash(string%(ref,srp),'spr_db','short')

9735491


In [4]:
ref = ws+'/Databases/Cattle_db/GCF_002263795.1.fna'
srp = ws+'/Databases/Cattle_db/GCF_002263795.srprism'
bmt = ws+'/Databases/Cattle_db/GCF_002263795.bitmask' 
!srprism mkindex -i Databases/Cattle_db/GCF_002263795.1.fna -o Databases/Cattle_db/GCF_002263795.bitmask -M 100000

/bin/bash: srprism: command not found


In [12]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=48:00:00
#PBS -l mem=15gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
cd %s
wget "ftp://ftp.ncbi.nlm.nih.gov/blast/db/nt.*.tar.gz"
for a in nt.*.tar.gz; do tar xzf $a; done
'''
BL = ws+'/Databases/NCBI_nt'
!mkdir $BL
bash(string%(BL),'NCBI_nt_db','short')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Databases/NCBI_nt’: File exists
9732615


In [11]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=48:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
cd %s
wget ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
tar -xvf taxdump.tar.gz
'''
Tx = ws+'/Databases/NCBI_tax'
!mkdir $Tx
bash(string%(Tx),'NCBI_tax_db','short')

9732614


In [1]:
!cp -r metaWRAP/bin/* ~/miniconda3/envs/metawrap/bin/

# ReadQC

In [84]:
string = '''
#PBS -l nodes=1:ppn=6
#PBS -l walltime=16:00:00
#PBS -l mem=12gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap read_qc -1 %s -2 %s -t 6 -o %s -x GCF_002263795
conda deactivate
'''
RQ = ws+'/ReadQC'
!mkdir $RQ
import os
for read in reads:
 #if read != '4':
 #    continue
    out = RQ+'/%s' % str(read)
    if os.path.exists(out): continue
    in1 = ws+'/Raw_reads/rstc.%s_1.fq.gz' % str(read)
    in2 = ws+'/Raw_reads/rstc.%s_2.fq.gz' % str(read)
    bash(string%(in1,in2,out),'%_ReadQC'%read,'short')

9736505


In [3]:
!mkdir $ws/ReadQC/stats
!mv $ws/ReadQC.sh.* $ws/ReadQC/stats/

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/ReadQC/stats’: File exists
mv: cannot stat ‘/beegfs/work/graaf20/Rusitec_meta/ReadQC.sh.*’: No such file or directory


In [50]:
# Check ReadQC outputs
import os
RQ = ws+'/ReadQC'
for read in reads:
    out = RQ+'/%s'%read
 #check output dir
    if os.path.exists(out) == False:
        print('ReadQC output is missing for sample: ',read)
    for file in ['final_pure_reads_1.fastq','final_pure_reads_2.fastq']:
 #check if pure reads in output dir
        if file not in os.listdir(out):
            print(file,'is missing for sample:',read)
 # check if size of pure reads is making sense
        size = round(os.stat(out+'/%s'%file).st_size*1e-9,3)
        if size < 16:
            print(read,file,size)

1 final_pure_reads_1.fastq 13.848
1 final_pure_reads_2.fastq 13.845
final_pure_reads_1.fastq is missing for sample: 4


FileNotFoundError: [Errno 2] No such file or directory: '/beegfs/work/graaf20/Rusitec_meta/ReadQC/4/final_pure_reads_1.fastq'

In [62]:
# gzip reads to save some space
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=3:00:00
#PBS -l mem=5gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
gzip %s/*.fastq
conda deactivate
'''
import os
RQ = ws+'/ReadQC'
for read in reads:
 #if read != '4':
 #    continue
    out = RQ+'/%s'%read
    bash(string%out,'%s_gzip'%read,'short')

9737128


# Pool samples by metadata

In [15]:
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=00:20:00
#PBS -l mem=5gb
#PBS -S /bin/bash

cp %s %s
cp %s %s
'''

import os
import pandas as pd

RQ = ws+'/ReadQC'
Pl = ws+'/Pooled'
!mkdir $Pl
md = pd.read_csv('metadata.tsv',index_col=0,sep='\t')
for read in reads:
    if read == '4':
        continue
    inp = RQ+'/%s'%read
    name = md.loc[int(read),'BS_Day_Trt_Src']
    inp1 = inp+'/final_pure_reads_1.fastq.gz'
    inp2 = inp+'/final_pure_reads_2.fastq.gz'
    out1 = Pl+'/'+read+'_'+name+'_1.fastq.gz'
    out2 = Pl+'/'+read+'_'+name+'_2.fastq.gz'
    bash(string%(inp1,out1,inp2,out2),'pool','tiny')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Pooled’: File exists
9776982
9776983
9776985
9776986
9776987
9776988
9776989
9776990
9776991
9776992
9776993
9776994
9776996
9776997
9776998
9776999
9777000
9777001
9777002
9777003
9777004
9777005
9777006
9777007
9777009
9777010
9777011
9777012
9777013
9777014
9777015
9777016
9777017
9777018
9777019
9777020
9777022
9777023
9777024
9777025
9777026
9777027
9777028
9777029
9777030
9777031
9777032
9777033
9777035
9777036
9777037


In [8]:
#concatenate all samples
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=04:00:00
#PBS -l mem=5gb
#PBS -S /bin/bash

cat %s/*_%s_*_1.fastq.gz > %s/%s_pooled_1.fastq.gz
cat %s/*_%s_*_1.fastq.gz > %s/%s_pooled_2.fastq.gz
'''
import pandas as pd
Pl = ws+'/Pooled'
md = pd.read_csv('metadata.tsv',index_col=0,sep='\t')
for bs in bss:
    bash(string%(Pl,bs,Pl,bs,Pl,bs,Pl,bs),'%s_pool'%bs,'short')
    


9786036
9786037
9786038
9786039


#    Assembling metagenomes

In [6]:
#!rm -r $ws/Metaspades_pooled $ws/Kraken2 $ws/Metaspades $ws/MegaHit $ws/Trimmed_paired_reads $ws/Trimmed_merged_reads
#!rm $ws/*.sh*

In [10]:
string = '''
#PBS -l nodes=1:ppn=%s
#PBS -l walltime=07:00:00:00
#PBS -l mem=%sgb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap

metaspades.py -k 21,33,55,77 -1 %s/%s_pooled_1.fastq.gz -2 %s/%s_pooled_2.fastq.gz -o %s -m %s -t %s
conda deactivate
'''
Pl = ws+'/Pooled'
MSp = ws+'/Metaspades_pooled'
!mkdir $MSp
for bs in bss:
    m, t = 1000, 40
    if bs in ['RSP','RF']:
        m, t = 500, 20
    out = MSp+'/'+bs
    bash(string%(t,m,Pl,bs,Pl,bs,out,m,t),'%s_MSpool'%bs,'smp')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Metaspades_pooled’: File exists
9908330


In [15]:
string = '''
#PBS -l nodes=1:ppn=%s
#PBS -l walltime=07:00:00:00
#PBS -l mem=%sgb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metaspades.py -k 21,33,55,77 -o %s -m %s -t %s --restart-from last
conda deactivate
'''
Pl = ws+'/Pooled'
MSp = ws+'/Metaspades_pooled'
for bs in bss:
    m, t = 1000, 40
    if bs in ['RSP','RF']: continue
    out = MSp+'/'+bs
    bash(string%(t,m,out,m,t),'%s_MSpool'%bs,'smp')

9904127
9904128


In [6]:
MS = ws+'/Metaspades'

import os
for read in reads:
    out = MS+'/%s' % read
    if 'final_assembly.fasta' not in os.listdir(out):
        !rm -r $out

### Quast on assemblies

In [21]:
string = '''
#PBS -l nodes=1:ppn=4
#PBS -l walltime=02:00:00
#PBS -l mem=32gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
quast -t 4 -o %s %s
conda deactivate
'''
MSp = ws+'/Metaspades_pooled'
for bs in bss:
 #if bs not in ['FL']: continue
    ass = MSp+'/%s/scaffolds.fasta'%bs
    out = MSp+'/'+bs+'/Quast'
    bash(string%(out,ass),'%s_Quast'%bs,'short')

9911366


In [6]:
string = '''
#PBS -l nodes=1:ppn=28
#PBS -l walltime=07:00:00:00
#PBS -l mem=128
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
metaspades.py -k 21,33,55,77 -1 %s -2 %s -o %s -m 128 -t 28
quast -t 28 -o %s %s
conda deactivate
'''
import os
Pl = ws+'/Pooled'
MS = ws+'/Metaspades'
!mkdir $MS
for read in reads:
    out = MS+'/'+read
    if read not in ['4']: continue
    if os.path.exists(out): continue
    r1 = Pl+'/%s_*_1.fastq.gz'%read
    r2 = Pl+'/%s_*_2.fastq.gz'%read
    ass = MS+'/%s/scaffolds.fasta'%read
    qua = MS+'/%s/Quast'%read
    bash(string%(r1,r2,out,ass,qua),'%s_MSpool'%read,'long')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Metaspades’: File exists
9925614


### Assemble sample by sample using Megahit

In [12]:
string = '''
#PBS -l nodes=1:ppn=28
#PBS -l walltime=48:00:00
#PBS -l mem=128
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
megahit -1 %s -2 %s -o %s -t 28 --presets meta-sensitive
quast -t 28 -o %s %s
conda deactivate
'''
import os
Pl = ws+'/Pooled'
MH = ws+'/Megahit'
!mkdir $MH
for read in reads:
    out = MH+'/'+read
    if read in ['4']: continue
    if os.path.exists(out): continue
    r1 = Pl+'/%s_*_1.fastq.gz'%read
    r2 = Pl+'/%s_*_2.fastq.gz'%read
    ass = MH+'/%s/final.contigs.fa'%read
    qua = MH+'/%s/Quast'%read
    bash(string%(r1,r2,out,qua,ass),'%s_MHit'%read,'short')

9925617


In [4]:
#print assembly size, gb
MH = ws+'/Megahit'
for read in reads:
    inp = MH+'/%s/final.contigs.fa' % read
 #if round(os.stat(inp).st_size * 9.5367431640625E-10,1) < 5:
    print(read, round(os.stat(inp).st_size * 9.5367431640625E-10,3))

1 1.354
4 1.187
9 0.438
11 0.503
13 0.508
15 0.523
17 0.513
19 0.518
21 0.422
23 0.581
25 0.491
27 0.607
29 0.488
31 0.506
44 0.743
45 0.718
46 0.773
47 0.729
48 0.735
49 0.649
50 0.822
51 0.838
52 0.764
53 0.686
54 0.733
55 0.678
80 1.453
83 1.355
88 0.411
90 0.507
92 0.511
94 0.506
96 0.551
98 0.619
100 0.417
102 0.482
104 0.591
106 0.509
108 0.525
110 0.571
123 0.684
124 0.69
125 0.747
126 0.719
127 0.75
128 0.838
129 0.876
130 0.839
131 0.797
132 0.698
133 0.787
134 0.854


In [5]:
#!rm $ws/*MHit.sh.*

# Kraken2

### By samples

In [61]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=08:00:00
#PBS -l mem=65gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap kraken2 -o %s -t 14 %s %s
conda deactivate
'''
Kr = ws+'/Kraken2_separate'
!mkdir $Kr
import os
for read in reads:
 #if read != '1': continue
    out = Kr+'/%s' % read
    if os.path.exists(out): continue
    in1 = ws+'/Megahit/%s/final.contigs.fa' % read
    in2 = ws+'/Pooled/%s_*.fastq.gz' % read
    bash(string%(out,in1,in2),'%s_Krk2'%read,'short')
    !mkdir $out

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Kraken2_separate’: File exists
9925945


In [152]:
#!rm -r Kraken2_separate/19
!rm *_Krk2.sh*

### Pooled

In [18]:
string = '''
#PBS -l nodes=1:ppn=24
#PBS -l walltime=24:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap kraken2 -o %s -t 24 %s %s
conda deactivate
'''

Kr = ws+'/Kraken2'
!mkdir $Kr
for bs in bss:
    out = Kr+'/%s' % bs
    if os.path.exists(out): continue
    in1 = ws+'/Metaspades_pooled/%s/scaffolds.fasta' % bs
    in2 = ws+'/Pooled/*_%s_d*.fastq.gz' % bs
    bash(string%(out,in1,in2),'%s_Krk2'%bs,'short')
    !mkdir $out

9911360
9911361
9911362
9911363


# Binning

### By samples

In [182]:
string = '''
#PBS -l nodes=1:ppn=24
#PBS -l walltime=24:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap binning -o %s -t 24 -a %s --metabat2 --maxbin2 --concoct %s
conda deactivate
'''
Bins = ws+'/Initial_Bins_separated'
!mkdir $Bins
for read in reads:
 #if read != '1': continue
    out = Bins+'/%s' % read
    if os.path.exists(out): continue
    in1 = ws+'/Megahit/%s/final.contigs.fa' % read
    in2 = ws+'/Pooled/%s_*.fastq.gz' % read
    bash(string%(out,in1,in2),'%s_bins'%read,'short')
    !mkdir $out

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Initial_Bins_separated’: File exists
9926514


In [47]:
#!rm -r Initial_Bins_separated/134
#!rm 134_bins.sh*

### Pooled

In [4]:
Bins = ws+'/Initial_Bins_pooled'
#!rm -r $Bins

In [43]:
string = '''
#PBS -l nodes=1:ppn=24
#PBS -l walltime=7:00:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap binning -o %s -t 24 -a %s --metabat2 --maxbin2 --concoct %s
conda deactivate
'''
Bins = ws+'/Initial_Bins_pooled'
!mkdir $Bins
for bs in bss:
    out = Bins+'/%s' % bs
    if os.path.exists(out): 
        continue
    in1 = ws+'/Metaspades_pooled/%s/scaffolds.fasta' % bs
    in2 = ws+'/Pooled/*_%s_d*.fastq.gz' % bs
    bash(string%(out,in1,in2),'%s_bins'%bs,'long')
    !mkdir $out

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Initial_Bins_pooled’: File exists


In [44]:
string = '''
#PBS -l nodes=1:ppn=28
#PBS -l walltime=7:00:00:00
#PBS -l mem=80gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap binning -o %s -t 28 -a %s --maxbin2 %s
conda deactivate
'''
Bins = ws+'/Initial_Bins_pooled'
!mkdir $Bins
for bs in bss:
    if bs != 'FL':continue
    out = Bins+'/%s' % bs
    in1 = ws+'/Metaspades_pooled/%s/scaffolds.fasta' % bs
    in2 = ws+'/Pooled/*_%s_d*.fastq.gz' % bs
    bash(string%(out,in1,in2),'%s_bins'%bs,'long')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Initial_Bins_pooled’: File exists
9925943
mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Initial_Bins_pooled/FL’: File exists


In [5]:
#!rm -rf $ws/Initial_Bins_*

# Bin_refinement

### By samples

In [28]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=24:00:00
#PBS -l mem=70gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap bin_refinement -o %s -t 14 -A %s/metabat2_bins/ -B %s/maxbin2_bins/ -C %s/concoct_bins/ -c 70 -x 10
conda deactivate
'''
BR = ws+'/Bins_refinement_separated_70_10'
!mkdir $BR
import os
for read in reads:
 #if read in ['134']: continue
    out = BR+'/%s' % read
    if os.path.exists(out): continue
    Bins = ws+'/Initial_Bins_separated/%s' % read
    bash(string%(out,Bins,Bins,Bins),'%s_bins_ref_70_10'%read,'short')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Bins_refinement_separated_70_10’: File exists
9927147


In [27]:
!rm -r Bins_refinement_separated_70_10/45
!rm 45_bins_ref_70_10.sh.*

#0 bins: 1, 
#low number bins: 80,83,4,

### Pooled

In [5]:
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=48:00:00
#PBS -l mem=80gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap bin_refinement -o %s -t 12 -A %s/metabat2_bins/ -B %s/maxbin2_bins/ -C %s/concoct_bins/ -c 70 -x 10
conda deactivate
'''
BR = ws+'/Bins_refinement_pooled_70_10'
!mkdir $BR
import os
for bs in bss:
 #if bs == 'FL':continue
    out = BR+'/%s' % bs
    if os.path.exists(out):continue
    bins = ws+'/Initial_Bins_pooled/%s' % bs
    bash(string%(out,bins,bins,bins),'%s_bins_ref'%bs,'short')

9929619
9929620
9929621
9929622


# Reassemble_bins

### By samples

In [33]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=48:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap reassemble_bins -o %s -1 %s -2 %s -t 14 -m 60 -c 70 -x 10 -b %s
conda deactivate
'''
BR = ws+'/Bins_refinement_separated_70_10'
RA = ws+'/Bins_reassembly_separated_70_10'
!mkdir $RA
import os
for read in reads:
 #if read not in ['1']: continue
    out = RA+'/%s' % read
    R1 = ws+'/Pooled/%s_*_1.fastq.gz' % read
    R2 = ws+'/Pooled/%s_*_2.fastq.gz' % read
    brs = BR+'/%s/metawrap_70_10_bins'% read
    if os.path.exists(out): continue
    bash(string%(out,R1,R2,brs),'%s_reassemble_70_10'%read,'short')

9928008
9928009
9928010
9928011
9928012
9928013
9928019
9928020
9928021
9928022
9928023
9928024
9928025
9928026
9928027
9928028
9928029
9928030
9928031
9928032
9928033
9928034
9928035
9928036
9928037
9928038
9928039
9928040
9928041
9928042
9928043
9928044
9928045
9928046
9928047
9928048
9928049
9928050
9928051
9928052
9928053
9928054
9928055
9928056
9928057
9928058
9928059
9928060
9928061
9928062
9928063
9928064


### Pooled

In [11]:
string = '''
#PBS -l nodes=1:ppn=24
#PBS -l walltime=7:00:00:00
#PBS -l mem=100gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap reassemble_bins -o %s -1 %s -2 %s -t 24 -m 100 -c 70 -x 10 -b %s
conda deactivate
'''
BR = ws+'/Bins_refinement_pooled_70_10'
RA = ws+'/Bins_reassembly_pooled_70_10'
!mkdir $RA
import os
for bs in bss:
 #if bs == 'FL':continue
    out = RA+'/%s' % bs
    R1 = ws+'/Pooled/%s_pooled_1.fastq.gz' % bs
    R2 = ws+'/Pooled/%s_pooled_2.fastq.gz' % bs
    brs = BR+'/%s/metawrap_70_10_bins'% bs
    if os.path.exists(out): continue
    bash(string%(out,R1,R2,brs),'%s_reassemble_70_10'%bs,'long')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Bins_reassembly_pooled_70_10’: File exists
9931897
9931898


# Assign taxonomy to bins

### By samples

In [35]:
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=24:00:00
#PBS -l mem=20gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap classify_bins -b %s -o %s -t 12
conda deactivate
'''
BC = ws+'/Bins_classification_separated_70_10'
RA = ws+'/Bins_reassembly_separated_70_10'
!mkdir $BC
import os
for read in reads:
 #if read not in ['1']: continue
    out = BC+'/%s' % read
    bra = RA+'/%s/reassembled_bins'% read
    if os.path.exists(out): continue
    bash(string%(bra,out),'%s_taxabins'%read,'short')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Bins_classification_separated_70_10’: File exists
9931557
9931558
9931559
9931560
9931561
9931562
9931563
9931564
9931565
9931566
9931567
9931568
9931569
9931570
9931571
9931572
9931573
9931574
9931575
9931576
9931577
9931578
9931579
9931580
9931581
9931582
9931583
9931584
9931585
9931586
9931587
9931588
9931589
9931590
9931591
9931592
9931593
9931594
9931595
9931596
9931597
9931598
9931599
9931600
9931601
9931602
9931603
9931604
9931605
9931606
9931607


### Pooled

In [4]:
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=4:00:00:00
#PBS -l mem=40gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metawrap classify_bins -b %s -o %s -t 16
conda deactivate
'''
BC = ws+'/Bins_classification_pooled_70_10'
RA = ws+'/Bins_reassembly_pooled_70_10'
!mkdir $BC
import os
for bs in bss:
    if bs in ['RSP','RF']: continue
    out = BC+'/%s' % bs
    bra = RA+'/%s/reassembled_bins'% bs
    if os.path.exists(out): continue
    bash(string%(bra,out),'%s_taxabins_pooled'%bs,'long')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Bins_classification_pooled_70_10’: File exists
9937393
9937394


# Functionally annotate bins

### By samples

In [98]:
string = '''
#PBS -l nodes=1:ppn=6
#PBS -l walltime=06:00:00
#PBS -l mem=20gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metaWRAP annotate_bins -o %s -t 6 -b %s
conda deactivate
'''
BA = ws+'/Bins_annotations_separated_70_10'
RA = ws+'/Bins_reassembly_separated_70_10'
!mkdir $BA
import os
for read in reads:
 #if read not in ['1']: continue
    out = BA+'/%s' % read
    bra = RA+'/%s/reassembled_bins'% read
    if os.path.exists(out): continue
    bash(string%(out,bra),'%s_anotbins'%read,'short')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Bins_annotations_separated_70_10’: File exists
9931986
9931987
9931988
9931989
9931990
9931991
9931992
9931993
9931994
9931995
9931996
9931997
9931998
9931999
9932000
9932001
9932002
9932003
9932004
9932005
9932006
9932007
9932008
9932009
9932010
9932011
9932012
9932013
9932014
9932015
9932016
9932017
9932018
9932019
9932020
9932021
9932022
9932023
9932024
9932025
9932026
9932027
9932028
9932029
9932030
9932031
9932032
9932033
9932034
9932035
9932036


### Pooled

In [8]:
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=4:00:00:00
#PBS -l mem=40gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate metawrap
export TMPDIR=$TMPDIR
metaWRAP annotate_bins -o %s -t 12 -b %s
conda deactivate
'''
BA = ws+'/Bins_annotations_pooled_70_10'
RA = ws+'/Bins_reassembly_pooled_70_10'
!mkdir $BA
import os
for bs in bss:
    if bs in ['RSP','RF']: continue
    out = BA+'/%s' % bs
    bra = RA+'/%s/reassembled_bins'% bs
    if os.path.exists(out): continue
    bash(string%(out,bra),'%s_anotbins_pooled'%bs,'long')

mkdir: cannot create directory ‘/beegfs/work/graaf20/Rusitec_meta/Bins_annotations_pooled_70_10’: File exists
9937397
9937398


In [3]:
#!rm -r Bins_annotations_pooled_70_10 Bins_annotations_separated_70_10
!rm *.sh.*

# Clean up!!!

In [5]:
# clean metaspades directory
keep = ['Quast','contigs.fasta','contigs.paths','scaffolds.fasta','scaffolds.paths','spades.log']
for bs in bss:
    for file in os.listdir('Metaspades_pooled/%s'%bs):
        if file not in keep:
            path = 'Metaspades_pooled/%s/%s'%(bs,file)
            !rm -rf $path

In [4]:
# clean megahit directory
keep = ['Quast','final.contigs.fa','log']
for read in reads:
    for file in os.listdir('Megahit/%s'%read):
        if file not in keep:
            path = 'Megahit/%s/%s'%(read,file)
            !rm -rf $path

In [2]:
!rm -rf HUMAnN MetaPhlan

In [1]:
!rm -rf Bins_* Kraken*

# SqueezeMeta

In [9]:
#!rm -rf *_SqueezeMeta_pooled

In [5]:
string = '''
#PBS -l nodes=1:ppn=%s
#PBS -l walltime=%s
#PBS -l mem=%sgb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
export TMPDIR=$TMPDIR
SqueezeMeta.pl -m coassembly -p %s -s %s -f Pooled -extassembly %s -extdb mydb.list -t %s
conda deactivate
'''

for bs in bss:
    if bs not in ['RSP','RF']: continue
    threads,time,mem,queue= 8,'7:00:00:00',64,'long'
    if bs in ['FL','FR']:
        threads,time,mem,queue= 24,'25:00:00:00',180,'smp'
    project = bs+'_SqueezeMeta_CAZy'
    samples = pd.DataFrame()
    for i,f in enumerate(sorted([x for x in os.listdir('Pooled') if '_%s_'%bs in x])):
        samples.loc[i,'sampleID'] = 'sample'+f.split('_')[0]
        samples.loc[i,'sampleName'] = f
        samples.loc[i,'pair'] = 'pair'+f.split('_')[-1].split('.')[0]
    samplesDF = '%s.tsv'%bs
    samples.to_csv(samplesDF, sep='\t',header=False,index=False)
    assembly = 'Metaspades_pooled/%s/scaffolds.fasta' % bs
    bash(string%(threads,time,mem,ws,project,samplesDF,assembly,threads),'%s_SqueezeMeta_CAZy'%bs,queue)

10127778


In [5]:
string = '''
#PBS -l nodes=1:ppn=%s
#PBS -l walltime=%s
#PBS -l mem=%sgb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
export TMPDIR=$TMPDIR
restart.pl %s
conda deactivate
'''

for bs in bss:
    if bs not in ['FL']: continue
    threads,time,mem,queue= 12,'8:00:00:00',128,'smp'
    if bs in ['FL','FR']:
        threads,time,mem,queue= 12,'07:00:00:00',120,'long'
    project = bs+'_SqueezeMeta_CAZy'
    bash(string%(threads,time,mem,ws,project),'%s_SqueezeMeta_restart'%bs,queue)

10197141


# dbCAN2

In [ ]:
string = '''
#PBS -l nodes=1:ppn=4
#PBS -l walltime=24:00:00
#PBS -l mem=16gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate dbcan
cd %s
run_dbcan.py [inputFile] [inputType] [-c AuxillaryFile] [-t Tools] 
conda deactivate
'''

for read in reads:
    for i,f in enumerate(sorted([x for x in os.listdir('Pooled') if '_%s_'%bs in x])):
        samples.loc[i,'sampleID'] = 'sample'+f.split('_')[0]
        samples.loc[i,'sampleName'] = f
        samples.loc[i,'pair'] = 'pair'+f.split('_')[-1].split('.')[0]
    samplesDF = '%s.tsv'%bs
    samples.to_csv(samplesDF, sep='\t',header=False,index=False)
    assembly = 'Metaspades_pooled/%s/scaffolds.fasta' % bs
    bash(string%(threads,time,mem,ws,project,samplesDF,assembly,threads),'%s_SqueezeMeta_CAZy'%bs,'short')

## Export SqueezeMeta outputs
## iTOL

In [12]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=02:00:00
#PBS -l mem=5gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
sqm2itol.pl -completion 50 -classification metacyc %s
cd %s/ext_tables
mkdir iTOL
mv *.txt iTOL
mv *.tar iTOL
mv *.nw iTOL
conda deactivate
'''

for bs in bss:
 #if bs not in ['FL']: continue
    project = bs+'_SqueezeMeta_CAZy'
    bash(string%(ws,project,project),'%s_SM_to_iTOL'%bs,'short')

10224448
10224449
10224450
10224451


## iPath

In [18]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=02:00:00
#PBS -l mem=5gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
sqm2ipath.pl %s
conda deactivate
'''

for bs in bss:
 #if bs not in ['FL']: continue
    project = bs+'_SqueezeMeta_CAZy'
    bash(string%(ws,project),'%s_SM_to_iPath'%bs,'short')

10224459
10224460
10224461
10224462


## Pavian

In [22]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=00:20:00
#PBS -l mem=4gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
sqm2pavian.pl RSP_SqueezeMeta_CAZy
mkdir RSP_SqueezeMeta_CAZy/ext_tables/Pavian
mv *.pavian RSP_SqueezeMeta_CAZy/ext_tables/Pavian
sqm2pavian.pl RF_SqueezeMeta_CAZy
mkdir RF_SqueezeMeta_CAZy/ext_tables/Pavian
mv *.pavian RF_SqueezeMeta_CAZy/ext_tables/Pavian
sqm2pavian.pl FR_SqueezeMeta_CAZy
mkdir FR_SqueezeMeta_CAZy/ext_tables/Pavian
mv *.pavian FR_SqueezeMeta_CAZy/ext_tables/Pavian
sqm2pavian.pl FL_SqueezeMeta_CAZy
mkdir FL_SqueezeMeta_CAZy/ext_tables/Pavian
mv *.pavian FL_SqueezeMeta_CAZy/ext_tables/Pavian
conda deactivate
'''
bash(string%(ws),'SM_to_Pavian','tiny')

10224468


## Export tables

In [24]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=04:00:00
#PBS -l mem=100gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
sqm2tables.py %s %s/ext_tables/Tables
conda deactivate
'''

for bs in bss:
 #if bs not in ['FL']: continue
    project = bs+'_SqueezeMeta_CAZy'
    bash(string%(ws,project,project),'%s_SM_tables'%bs,'short')

10224471
10224472
10224473
10224474


In [25]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=04:00:00
#PBS -l mem=100gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
sqm2tables.py --ignore-unclassified %s %s/ext_tables/Tables_no_uncl
conda deactivate
'''

for bs in bss:
 #if bs in ['FL','FR']: continue
    project = bs+'_SqueezeMeta_CAZy'
    bash(string%(ws,project,project),'%s_SM_tables_nouncl'%bs,'short')

10224475
10224476
10224477
10224478


## Anvio

In [55]:
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=02:00:00
#PBS -l mem=20gb
#PBS -S /bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate SqueezeMeta
cd %s
sqm2anvio.pl %s %s/ext_tables/Anvio 7
conda deactivate
'''

for bs in bss:
 #if bs in ['FL','RSP']: continue
    project = bs+'_SqueezeMeta_CAZy'
    bash(string%(ws,project,project),'%s_SqueezeMeta_anvio'%bs,'short')

10249290
10249291
10249292
10249293


# Pigz

In [82]:
string = '''
#PBS -l nodes=1:ppn=4
#PBS -l walltime=04:00:00
#PBS -l mem=12gb
#PBS -S /bin/bash
cd %s
mkdir %s
cp -r %s/ext_tables %s/results %s/
tar -c --use-compress-program=pigz -f %s.tar %s
rm -rf %s
'''

for bs in bss:
 #if bs in ['FL','RSP']: continue
    archive = bs+'_SM_CAZy_archive'
    project = bs+'_SqueezeMeta_CAZy'
    bash(string%(ws,archive,project,project,archive,archive,archive,archive),'%s_archive'%bs,'short')

10249370
10249371
10249372
10249373


In [1]:
!du -sh *Squeeze*

571G	FL_SqueezeMeta_coassembly
568G	FR_SqueezeMeta_coassembly
52G	RF_SqueezeMeta_coassembly
54G	RSP_SqueezeMeta_coassembly


In [1]:
!qstat -u ho_graaf20


mgmt02: 
                                                                                  Req'd       Req'd       Elap
Job ID                  Username    Queue    Jobname          SessID  NDS   TSK   Memory      Time    S   Time
----------------------- ----------- -------- ---------------- ------ ----- ------ --------- --------- - ---------
10954217                ho_graaf20  long     SM_tables.sh       4823     1      2     100gb  04:00:00 C       -- 
10954218                ho_graaf20  long     SM_tables.sh       5778     1      2     100gb  04:00:00 R  00:00:44


In [80]:
!qdel 10249345 10249346 10249347 10249348

In [8]:
!qselect -q long | xargs qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
10130727                   ACE_Biocat       st_ac125867            0 Q long           
10136967                   default          tu_zxmcy32      41:20:07 R long           
10136972                   default          tu_zxmcy32      226:41:3 R long           
10136978                   default          tu_zxmcy32      51:35:35 R long           
10136979                   default          tu_zxmcy32      86:05:51 R long           
10136983                   default          tu_zxmcy32      100:53:4 R long           
10136986                   default          tu_zxmcy32      83:49:03 R long           
10136987                   default          tu_zxmcy32      89:27:45 R long           
10136988                   default          tu_zxmcy32      29:45:06 R long           
10136990                   default          tu_zxmcy32      95:23

In [2]:
for bs in bss:
        !rm -r $bs*/data/sam
        !rm -r $bs*/data/raw_fastq

In [3]:
!zip -r SM_MAGs_Rusitec.zip 

Scanning files 
  adding: FL_SqueezeMeta_CAZy/ (stored 0%)
  adding: FL_SqueezeMeta_CAZy/.ipynb_checkpoints/ (stored 0%)
  adding: FL_SqueezeMeta_CAZy/.ipynb_checkpoints/syslog-checkpoint (deflated 92%)
  adding: FL_SqueezeMeta_CAZy/results/ (stored 0%)
  adding: FL_SqueezeMeta_CAZy/results/06.FL_SqueezeMeta_CAZy.fun3.tax.wranks (deflated 92%)
  adding: FL_SqueezeMeta_CAZy/results/21.FL_SqueezeMeta_CAZy.kegg.pathways (deflated 86%)
  adding: FL_SqueezeMeta_CAZy/results/02.FL_SqueezeMeta_CAZy.trnas (deflated 76%)
  adding: FL_SqueezeMeta_CAZy/results/07.FL_SqueezeMeta_CAZy.fun3.pfam (deflated 76%)
  adding: FL_SqueezeMeta_CAZy/results/12.FL_SqueezeMeta_CAZy.CAZyDB.funcover (deflated 66%)
  adding: FL_SqueezeMeta_CAZy/results/DAS/ (stored 0%)
  adding: FL_SqueezeMeta_CAZy/results/DAS/FL_SqueezeMeta_CAZy_proteins.faa.bacteria.scg (deflated 85%)
  adding: FL_SqueezeMeta_CAZy/results/DAS/FL_SqueezeMeta_CAZy_DASTool_scg.log (deflated 87%)
  adding: FL_SqueezeMeta_CAZy/results/DAS/metabat2.ta

In [4]:
!zip -r samples.zip Pooled

  adding: Pooled/ (stored 0%)
  adding: Pooled/90_FR_d13_Trt1_HP2_2.fastq.gz (deflated 0%)
  adding: Pooled/129_FL_d13_Trt4_HP1_2.fastq.gz (deflated 0%)
  adding: Pooled/23_FR_d13_Trt4_HP2_2.fastq.gz (deflated 0%)
  adding: Pooled/51_FL_d13_Trt4_HP2_2.fastq.gz (deflated 0%)
  adding: Pooled/51_FL_d13_Trt4_HP2_1.fastq.gz (deflated 0%)
  adding: Pooled/15_FR_d13_Trt2_HP2_1.fastq.gz (deflated 0%)
  adding: Pooled/25_FR_d13_Trt5_HP1_1.fastq.gz (deflated 0%)
  adding: Pooled/49_FL_d13_Trt3_HP2_1.fastq.gz (deflated 0%)
  adding: Pooled/100_FR_d13_Trt4_HP1_1.fastq.gz (deflated 0%)
  adding: Pooled/17_FR_d13_Trt3_HP1_1.fastq.gz (deflated 0%)
  adding: Pooled/4_RF_d0_not_appl_C1_2.fastq.gz (deflated 0%)
  adding: Pooled/133_FL_d13_Trt6_HP1_2.fastq.gz (deflated 0%)
  adding: Pooled/50_FL_d13_Trt4_HP1_1.fastq.gz (deflated 0%)
  adding: Pooled/129_FL_d13_Trt4_HP1_1.fastq.gz (deflated 0%)
  adding: Pooled/49_FL_d13_Trt3_HP2_2.fastq.gz (deflated 0%)
  adding: Pooled/124_FL_d13_Trt1_HP2_1.fastq.gz (d

In [15]:
string = '''
#PBS -l nodes=1:ppn=1
#PBS -l walltime=02:00:00
#PBS -l mem=1gb
#PBS -S /bin/bash
cd %s
rm -rf *_coassembly
'''
bash(string%(ws),'Clean','short')

10224452


# Stat. analysis

In [ ]:
import pandas as pd
